In [1]:
library(tidyverse)
library(data.table)
library(readxl)
library(dplyr)
library(stringr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




Make a bedfile of the non-suspicious indels where the second position is length of ref - 1. 

In [2]:
## This script is to get a bedfile for rAF-lo IGM indels 

## The columns for the bedfiles are: CHR | START | END

setwd ("IGM/")

rAFlo_bp10 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp10_rAF_loIndels.lt50bp.csv")
rAFlo_bp20 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp20_rAF_loIndels.lt50bp.csv")
rAFlo_bp30 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp30_rAF_loIndels.lt50bp.csv")
rAFlo_bp40 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp40_rAF_loIndels.lt50bp.csv")

df_name = "IGM"


bp_range = c("10", "20", "30", "40")

for (i in bp_range) {
  get_df = paste0("rAFlo_bp", i)
  
  bed = paste0("bed_bp", i)
  
  assign(get_df, get(get_df) %>% separate(VarID, c("CHR", "POS", "REF", "ALT")))
  assign(get_df, get(get_df) %>% mutate(refLen = nchar(get(get_df)$REF)))
  assign(bed, data.frame(get(get_df)$CHR, get(get_df)$POS, as.numeric(get(get_df)$POS) + as.numeric(get(get_df)$refLen) - 1))

  fwrite(get(bed), paste0(df_name, "_rAF_lo_indels_bp", i, ".bed"), col.names=FALSE, sep = "\t")
}


Merge bedfile of gnomAD bins that are sAF-hi and rAF-hi. 
Bash script below. 

In [3]:
system("bp_range=('10' '20' '30' '40')

date='2023-09-12'
output_path='/gnomAD/'

df_name='gnomAD'

for i in ${bp_range[@]};
do
   echo $i
   cat ${output_path}gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp${i}_rAF_hiIndels.lt50bp.region.bed ${output_path}gnomad.exomes.r2.1.1.sites_indelsonly_rAF_bp${i}_sAF_hiIndels.lt50bp.region.bed > ${output_path}${df_name}_rAFhi_and_sAFhi_bins_bp${i}.bed
done;", intern = TRUE)

[1] "10" "20" "30" "40"

Bedtools intersect IGM indels that are rAF-lo by IGM indel with gnomAD bins that are sAF-hi or rAF-hi. This would combine the regions that are also sAF-hi or rAF-hi by gnomAD.

Bash script below.

In [4]:
system("bp_range=('10' '20' '30' '40')

date='2023-09-12'
output_path_df2='/IGM/'
output_path_df1='/gnomAD/'


df_1_name='gnomAD'
df_2_name='IGM'

for i in ${bp_range[@]};
do
   echo $i
   bedtools intersect -wa -a ${output_path_df2}${df_2_name}_rAF_lo_indels_bp${i}.bed -b ${output_path_df1}${df_1_name}_rAFhi_and_sAFhi_bins_bp${i}.bed | sort -k1,1n -k2,2n | uniq > ${output_path_df2}${df_2_name}_rAF_lo_indels_annotated_bp${i}.bed
done;", intern = TRUE)


[1] "10" "20" "30" "40"

Get files with: 
 1. File 1 (all_sus): [indels that are sus by IGM rAF] + [indels that are overlapping with gnomAD sus and common bins]
 2. File 2 (reduced_sus): [indels that are non-sus by IGM rAF] - [indels that are overlapping with gnomAD sus and common bins]

In [5]:
library(tidyverse)
library(data.table)
library(readxl)
library(dplyr)
library(stringr)

setwd ("/IGM")


## This script is to get a file with all the rAFhi IGM indels, including the indels that are rAFlo by IGM rAF but are common/rAFhi by gnomAD rAF 

bp_range = c("10", "20", "30", "40")
output_path = ""

df_name = "IGM"

###############################################################
###############################################################

## Read in IGM indels that are rAFhi by IGM rAF only 
rAF_hi_bp10 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp10_rAF_hiIndels.lt50bp.csv")
rAF_hi_bp20 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp20_rAF_hiIndels.lt50bp.csv")
rAF_hi_bp30 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp30_rAF_hiIndels.lt50bp.csv")
rAF_hi_bp40 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp40_rAF_hiIndels.lt50bp.csv")

## Read in rAFlo IGM indels 
rAF_lo_bp10 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp10_rAF_loIndels.lt50bp.csv")
rAF_lo_bp20 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp20_rAF_loIndels.lt50bp.csv")
rAF_lo_bp30 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp30_rAF_loIndels.lt50bp.csv")
rAF_lo_bp40 = fread("2023-03-23_IGM_n39367_indelsonly_rAF_bp40_rAF_loIndels.lt50bp.csv")

## Read in rAFlo IGM indels bedfile that are annotated to be non-rAFlo by gnomAD rAF and sAF
annotated_bp10 = fread("IGM_rAF_lo_indels_annotated_bp10.bed")
annotated_bp20 = fread("IGM_rAF_lo_indels_annotated_bp20.bed")
annotated_bp30 = fread("IGM_rAF_lo_indels_annotated_bp30.bed")
annotated_bp40 = fread("IGM_rAF_lo_indels_annotated_bp40.bed")

###############################################################
###############################################################

## Map the annotated_bp indels to the rAF_lo_bp dataframes by CHR and first POS

for (i in bp_range) {
  rAF_lo_bp = paste0("rAF_lo_bp", i)
  annotated_bp = paste0("annotated_bp", i)
  
  varID_annotated_bp = paste0("varID_annotated_bp", i)
  
  assign(rAF_lo_bp, get(rAF_lo_bp) %>% separate(VarID, c("CHR", "POS", "REF", "ALT")))
  
  assign(varID_annotated_bp, get(rAF_lo_bp) %>% filter(paste0(get(rAF_lo_bp)$CHR, "-", get(rAF_lo_bp)$POS) %in% paste0(get(annotated_bp)$V1, "-", get(annotated_bp)$V2)))
}

## Combine the rAFhi indels that are rAFhi by IGM rAF with the annotated rAFlo indels that are rAFhi by gnomAD rAF 

for (i in bp_range) {
  varID_annotated_bp = paste0("varID_annotated_bp", i)
  assign(varID_annotated_bp, subset((get(varID_annotated_bp) %>% mutate(VarID = paste0(CHR, "-", POS, "-", REF, "-", ALT))), select = -c(CHR, POS, REF, ALT)))
  
  ## bind to rAFhi indels by IGM rAF
  rAF_hi_bp = paste0("rAF_hi_bp", i)
  
  ## all rAFhi (including annotated one)
  all_rAF_hi_bp = paste0("all_rAF_hi_bp", i)
  
  assign(all_rAF_hi_bp, rbind(get(varID_annotated_bp), get(rAF_hi_bp)))
  fwrite(get(all_rAF_hi_bp), paste0(output_path, df_name, "_all_rAF_hi_bp", i, ".csv"))
}

## Find the reduced rAF_lo indels.
## rAFlo indels that are NOT in gnomAD bins that are common/rAF_hi. 
for (i in bp_range) {
  rAF_lo_bp = paste0("rAF_lo_bp", i)
  assign(rAF_lo_bp, subset((get(rAF_lo_bp) %>% mutate(VarID = paste0(CHR, "-", POS, "-", REF, "-", ALT))), select = -c(CHR, POS, REF, ALT)))
  
  varID_annotated_bp = paste0("varID_annotated_bp", i)
  reduced_rAF_lo_bp = paste0("reduced_rAF_lo_bp", i)
  
  # exclude VarIDs that are in varID_annotated_bp from the rAF_lo_bp df 
  assign(reduced_rAF_lo_bp, get(rAF_lo_bp) %>% filter(!(VarID %in% get(varID_annotated_bp)$VarID)))
  fwrite(get(reduced_rAF_lo_bp), paste0(output_path, df_name, "_reduced_rAF_lo_bp", i, ".csv"))
  
}

Find the number of individuals that contain only suspicious deleterious indels by not counting the individuals that are also in the cohort with at least one suspicious deleterious indel in a constrained gene associated with an autosomal dominant disorder. 


In [6]:
## Table S10.

setwd ("/IGM")

library(tidyverse)
library(data.table)
library(readxl)
library(dplyr)
library(stringr)

bp_range = c("10", "20", "30", "40")

total_samples = 39367

variants_effects = c("frameshift_variant", "splice_donor_variant", "splice_acceptor_variant", "stop_gained", "start_lost", "stop_lost", "exon_loss_variant")

## This script finds the number of individuals with only deleterious rAF_hi indels in constrained, autosomal dominant genes  
all_rAF_hi_bp10 = fread("IGM_all_rAF_hi_bp10.csv")
all_rAF_hi_bp20 = fread("IGM_all_rAF_hi_bp20.csv")
all_rAF_hi_bp30 = fread("IGM_all_rAF_hi_bp30.csv")
all_rAF_hi_bp40 = fread("IGM_all_rAF_hi_bp40.csv")

## Read in rAF_lo IGM indels (removed the indels that are in common/rAF_hi gnomAD bins)
reduced_rAF_lo_bp10 = fread("IGM_reduced_rAF_lo_bp10.csv")
reduced_rAF_lo_bp20 = fread("IGM_reduced_rAF_lo_bp20.csv")
reduced_rAF_lo_bp30 = fread("IGM_reduced_rAF_lo_bp30.csv")
reduced_rAF_lo_bp40 = fread("IGM_reduced_rAF_lo_bp40.csv")

sample_name_gene_name = fread("2023-03-24_11-47-14_IGM_n39367_indels_genotypes_selectcols.csv") 
colnames(sample_name_gene_name) = c("VarID", "geneName", "sampleName", "coveredCtrl", "AC")


effects = fread("2023-04-21_IGM_n39367_indels_genotypes_effects.csv", header = TRUE)
colnames(effects) = c("VarID", "Effect")

annotations = distinct(fread ("2023-04-21_IGM_genename_gnomadpli_gnomadloeuf_omimdisease.csv")) #18231 
colnames(annotations) = c("geneName", "pLI", "oe_lof_upper", "OMIM_disease")

annotations[geneName == "'HTT'"]$OMIM_disease = "Huntington disease, 143100 (3), Autosomal dominant"
annotations[geneName == "'GLTSCR1'"]$geneName = "BICRA"
annotations[geneName == "'FAM46A'"]$geneName = "TENT5A"


## Merge IGM all rAF_hi indels with gene name, sample name, pLI, loeuf, omim, effects

for (i in bp_range) {
  all_rAF_hi_bp = paste0("all_rAF_hi_bp", i)
  
  rAF_hi_with_gene_bp = paste0("rAF_hi_with_gene_bp", i)
  rAF_hi_with_effects_bp = paste0("rAF_hi_with_effects_bp", i)
  rAF_hi_with_annot_bp = paste0("rAF_hi_with_annot_bp", i)
  
  assign (rAF_hi_with_gene_bp, merge(get(all_rAF_hi_bp), sample_name_gene_name, by="VarID" ))
  assign (rAF_hi_with_effects_bp, merge(get(rAF_hi_with_gene_bp), effects, by="VarID"))
  assign (rAF_hi_with_annot_bp, merge(get(rAF_hi_with_effects_bp), annotations, by="geneName"))
  ## rAF_hi dataframe with all the annotations is rAF_hi_with_annot_bp
  
  ## Now only look for rAF_hi indels of interest by filtering for indels that are autosomal dominant, deleterious, and constrained (pLI > 0.5; oe_lof_upper < 0.35)
  filtered_rAF_hi_bp = paste0("filtered_rAF_hi_bp", i)
  assign(filtered_rAF_hi_bp, get(rAF_hi_with_annot_bp)[OMIM_disease %like% "Autosomal dominant", ] %>% filter(Effect %in% variants_effects) %>% filter(oe_lof_upper < 0.35) %>% filter(pLI > 0.5))
  
  ### Now look for rAF_lo indels 
  
  reduced_rAF_lo_bp = paste0("reduced_rAF_lo_bp", i)
  
  rAF_lo_with_gene_bp = paste0("rAF_lo_with_gene_bp", i)
  rAF_lo_with_effects_bp = paste0("rAF_lo_with_effects_bp", i)
  rAF_lo_with_annot_bp = paste0("rAF_lo_with_annot_bp", i)
  ## reduced rAF_lo dataframe with all the annotations is rAF_lo_with_annot_bp
  
  assign(rAF_lo_with_gene_bp, merge(get(reduced_rAF_lo_bp), sample_name_gene_name, by = "VarID"))
  assign(rAF_lo_with_effects_bp, merge(get(rAF_lo_with_gene_bp), effects, by = "VarID"))
  assign(rAF_lo_with_annot_bp, merge(get(rAF_lo_with_effects_bp), annotations, by ="geneName"))
  
  ## Now only look for non rAF_hi indels of interest by filtering for indels that are autosomal dominant, deleterious, and constrained (pLI > 0.5; oe_lof_upper < 0.35)
  filtered_rAF_lo_bp = paste0("filtered_rAF_lo_bp", i)
  assign(filtered_rAF_lo_bp, get(rAF_lo_with_annot_bp)[OMIM_disease %like% "Autosomal dominant", ] %>% filter(Effect %in% variants_effects) %>% filter(oe_lof_upper < 0.35) %>% filter(pLI > 0.5))
  
  ## find unique individuals that are in filtered_rAF_lo_bp 
  rAF_lo_individuals = paste0("only_rAF_hi_individuals_bp", i)
  num_rAF_lo_individuals= paste0("num_rAF_lo_individuals_bp", i)
  
  assign(rAF_lo_individuals, as.data.frame(unique(get(filtered_rAF_lo_bp)$sampleName)))
  assign(num_rAF_lo_individuals, nrow(get(rAF_lo_individuals)))
  
  ## find unique rAF_hi individuals that are in filtered_rAF_lo_bp 
  ## this means rAF_hi indels that are NOT in non_rAF_hi indels 
  only_rAF_hi_individuals = paste0("only_rAF_hi_individuals_bp", i )
  num_only_rAF_hi_individuals = paste0("num_only_rAF_hi_individuals_bp", i)
  
  assign(only_rAF_hi_individuals, as.data.frame(unique((get(filtered_rAF_hi_bp) %>% filter(!(get(filtered_rAF_hi_bp)$sampleName %in% get(filtered_rAF_lo_bp)$sampleName)))$sampleName)) )
  assign(num_only_rAF_hi_individuals, nrow(get(only_rAF_hi_individuals)))
}

## make dataframe to visualize data 
rAF_lo_individuals = c(num_rAF_lo_individuals_bp10, num_rAF_lo_individuals_bp20, num_rAF_lo_individuals_bp30, num_rAF_lo_individuals_bp40)
only_rAF_hi_individuals = c(num_only_rAF_hi_individuals_bp10, num_only_rAF_hi_individuals_bp20, num_only_rAF_hi_individuals_bp30, num_only_rAF_hi_individuals_bp40)
  
summary = data.frame(bp_range, rAF_lo_individuals, only_rAF_hi_individuals)

## Find proportion out of all IGM cohort 
summary$prct_rAF_lo_out_of_total = round((as.numeric(summary$rAF_lo_individuals) / total_samples) * 100, 2) 
summary$prct_only_rAF_hi_out_of_total = round((as.numeric(summary$only_rAF_hi_individuals) / total_samples) * 100, 2) 
summary$prct_only_rAF_hi_out_of_rAF_lo = round((as.numeric(summary$only_rAF_hi_individuals) / (as.numeric(summary$rAF_lo_individuals))) * 100, 2) 
summary$prct_only_rAF_hi_out_of_5299 = round((as.numeric(summary$only_rAF_hi_individuals) / (5299)) * 100, 2) 


summary 

setwd("")
fwrite(summary, "tableS10.csv")

bp_range,rAF_lo_individuals,only_rAF_hi_individuals,prct_rAF_lo_out_of_total,prct_only_rAF_hi_out_of_total,prct_only_rAF_hi_out_of_rAF_lo,prct_only_rAF_hi_out_of_5299
<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
10,4063,1236,10.32,3.14,30.42,23.33
20,3790,1509,9.63,3.83,39.82,28.48
30,3569,1730,9.07,4.39,48.47,32.65
40,3373,1926,8.57,4.89,57.10,36.35
